In [1]:
from tqdm import tqdm
from collections import Counter

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from num2words import num2words
from collections import Counter
from scipy import spatial
import joblib
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from scipy.special import gammaln
from collections import Counter

import imp, multiprocessing
import datetime
import LDA_ETM as lda
import scipy
import operator
import nltk
import os
import string
import numpy as np
import copy
import pandas as pd
import pickle
import utils as my_utils
from sklearn.metrics import silhouette_score, davies_bouldin_score

In [2]:
dataset_name = "amazon_kindle_20000"

In [3]:
dataset = pd.read_pickle("datasets/"+ dataset_name + "_dataset")

In [4]:
n_sentiment = dataset.sentiment.unique().shape[0]

### Making Edge_Dict

In [5]:
embedding_name = "glove_0.6"

In [6]:
docs_edges = pickle.load(open("resources/"+ dataset_name + "_" +embedding_name + ".pickle","rb"))

## ETM

In [7]:
min_df = 5
max_df = .5
maxIters = 20

beta=0.1
alpha=0.1
n_topics = 5
lambda_param = 1.0

In [8]:
imp.reload(lda)

<module 'LDA_ETM' from '/home/william18026/Embedding-LJST-Paper/LDA_ETM.py'>

In [9]:
sampler = lda.LdaSampler(n_topics=n_topics, min_df=min_df, max_df=max_df, lambda_param=lambda_param,
                         alpha=alpha, beta=beta)

In [10]:
%%time
sampler._initialize_(reviews = dataset.text.tolist())

4936 (20000, 4936)
CPU times: user 4.34 s, sys: 576 ms, total: 4.92 s
Wall time: 4.91 s


In [11]:
sampler.run(edge_dict=docs_edges, maxiter=20, debug=True)

  0%|          | 52/20000 [00:00<00:38, 514.05it/s]

Iteration 0


  0%|          | 52/20000 [00:00<00:38, 514.94it/s]

Iteration 1


100%|██████████| 20000/20000 [00:51<00:00, 385.80it/s]


In [ ]:
joblib.dump(sampler, "dumps/mrf_lda" + dataset_name + "_" + embedding_name)

# Evaluations

In [ ]:
folder_name

In [ ]:
n_top_words = 5

In [ ]:
def func_eval(k):
    sampler = joblib.load("dumps/"+folder_name + "/sampler_n_topics_" + str(k))

    t_words = sampler.getTopKWords(n_top_words, words)
    top_words = [t_words[i] for i in t_words.keys()]
    dt_distribution = sampler.theta()

    print my_utils.coherence_score(count_matrix, top_words, vocabulary), "\t", silhouette_score(count_matrix, dt_distribution.argmax(axis=1)), "\t", davies_bouldin_score(count_matrix, dt_distribution.argmax(axis=1))

In [ ]:
for k in [lambda_param]:
    func_eval(k)

# Appendix

In [ ]:
    
#     print ""
#     print "Topics:", k
#     print "Coherance:", my_utils.coherence_score(count_matrix, top_words, vocabulary)
#     print "Silhouette Score:", silhouette_score(count_matrix, dt_distribution.argmax(axis=1))
#     print "Davies Bouldin Score:", davies_bouldin_score(count_matrix, dt_distribution.argmax(axis=1))


In [ ]:
# dt_dist = sampler.theta()

# X_embedded = TSNE(n_components=2).fit_transform(dt_dist)

# import matplotlib.pyplot as plt
# from sklearn.manifold import TSNE

# plt.figure(figsize=(8, 8))
# plt.scatter([i[0] for i in X_embedded], [i[1] for i in X_embedded], c=dt_dist.argmax(axis=1))
# plt.legend(loc=2)
# plt.show()